In [ ]:
import pandas as pd
from WindPy import w
import datetime as dt
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import json

In [ ]:
w.start()

## 导入数据，定义函数

In [ ]:
plt.rcParams['font.sans-serif'] = ['SimHei']  # 中文字体设置-黑体
plt.rcParams['axes.unicode_minus'] = False  # 解决保存图像是负号'-'显示为方块的问题
sns.set(font='SimHei',font_scale=1.5)  # 解决Seaborn中文显示问题并调整字体大小

In [ ]:
## 定义根据债券余额加权的点乘积：
def weighted_premium(dff_VS_GK):
    weighted_premium=np.dot(dff_VS_GK["券种利差"],dff_VS_GK["债券余额\n[日期] 最新\n[单位] 亿"]/dff_VS_GK["债券余额\n[日期] 最新\n[单位] 亿"].sum())
    return weighted_premium.round(2)

def get_credit_premium():
    data= pd.read_excel("Credit_Assistant.xlsx",skiprows=1,index_col=0).iloc[2:,:]
    index_code=pd.read_excel("Credit_Assistant.xlsx",skiprows=1,index_col=0).iloc[1,:].tolist()
    index_name=pd.read_excel("Credit_Assistant.xlsx").iloc[0,1:].tolist()
    str=","
    err,df=w.edb(str.join(index_code),"2019-01-01", dt.datetime.today().strftime("%Y-%m-%d"),"Fill=Previous",usedf=True)
    df.columns=index_name
    return df

In [ ]:
def weekly_change():
    data.index=pd.to_datetime(data.index)
    weekly_data = data.resample('W').mean()
    weekly_chg = weekly_data.diff(1).round(2)
    

In [ ]:
data= pd.read_excel("Credit_Assistant.xlsx",sheet_name="地产债",index_col=0)


In [ ]:
#读取城投债数据
io=r"城投债数据_t.xlsx"
data=pd.read_excel(io)

In [ ]:
## 筛选非PPN
data = data[data["证券简称"].str.contains("PPN")==False]
#确定债券的可比期限
data["含权债行权期限"]=data["含权债行权期限"].fillna(10)
data["含权债行权期限"]=data["含权债行权期限"]*365
df=pd.DataFrame(data=data)
df["期限"]=((data[["含权债行权期限","剩余期限(天)\n[日期] 最新\n[单位] 天"]].min(axis=1))/365).round(2)
df["期限_匹配"]=((data[["含权债行权期限","剩余期限(天)\n[日期] 最新\n[单位] 天"]].min(axis=1))/365).round(0)

In [ ]:
#获取最新的国开收益率基准
def GK_updated_yield(date=dt.datetime.today().strftime("%Y-%m-%d")):
    w.start()
    GK=pd.read_excel("Credit_Assistant.xlsx",sheet_name="国开可比基准",skiprows=1,index_col=0).iloc[2:,:]
    GK_index_name=GK.columns.tolist()
    str=","

    GK_index_code=str.join(pd.read_excel("Credit_Assistant.xlsx",sheet_name="国开可比基准",skiprows=1,index_col=0).iloc[1,:].tolist())

    err,GK_yield=w.edb(GK_index_code, "2020-7-1", date,"Fill=Previous",usedf=True)
    GK_yield.columns=GK_index_name

    df=GK_yield.tail(1).T
    df.columns=["GK_yield"]
    df["期限"]=[1,2,3,4,5]

    return df
    

In [ ]:
def get_credit_vs_gk_data():
    GK_yield_base=GK_updated_yield()
    dff_VS_GK=pd.merge(df[df["期限"]<5],GK_yield_base,left_on=["期限_匹配"],right_on=["期限"],how="left")
    dff_VS_GK["券种利差"]=(dff_VS_GK["债券估值(YY)\n[单位] %"]-dff_VS_GK["GK_yield"])*100
    dff_VS_GK=dff_VS_GK[dff_VS_GK["券种利差"].isna()==False]
    return dff_VS_GK

In [ ]:
dff_VS_GK=get_credit_vs_gk_data()

In [ ]:
## 主体债券维度：获取主体的债券维度与科目

statistic_method=np.median
info_dimension="证券代码","证券简称","期限","估价收益率(%)(中债)\n[日期] 最新收盘日\n[估值类型] 推荐","债券估值(YY)\n[单位] %","区域","城市",
loaner_name="青岛国信发展(集团)有限责任公司"

#def get_loaner_info(loaner_name,info_dimension,loaner_name): 
data_loaner=dff_VS_GK.groupby("主体名称")[info_dimension].get_group(loaner_name)


In [ ]:
city_name="青岛市"
info_dimension="券种利差","债券余额\n[日期] 最新\n[单位] 亿"
agg_dimension="城市"
## 求城市的利差
#def 集合利利差

city_credit_premium=dff_VS_GK.groupby("城市")[info_dimension].apply(lambda x : weighted_premium(x))
province_credit_premium=dff_VS_GK.groupby("区域")[info_dimension].apply(lambda x : weighted_premium(x))

In [ ]:
info_dimension="证券代码","证券简称","期限","估价收益率(%)(中债)\n[日期] 最新收盘日\n[估值类型] 推荐","债券估值(YY)\n[单位] %","债券余额\n[日期] 最新\n[单位] 亿","券种利差",
city_name="青岛市"

dff_VS_GK.groupby("城市")[info_dimension].get_group(city_name)

## 制作地理位置图

In [ ]:
import json
json_io=r"geojson-map-china\china.json"
gs_data = open(json_io, encoding='utf8').read()
gs_data = json.loads(gs_data)
#整理plotly需要的格式：
for i in range(len(gs_data["features"])):
    gs_data["features"][i]["id"]=gs_data["features"][i]["properties"]["id"]#id前置
    gs_data["features"][i]["name"]=gs_data["features"][i]["properties"]["name"]

### 匹配id和区域
geo_id=[]
geo_name=[]
for i in range(len(gs_data["features"])):
    geo_id.append(gs_data["features"][i]["id"])
    geo_name.append(gs_data["features"][i]['properties']["name"])
geo_data=pd.DataFrame({"id":geo_id,"区域":geo_name})

In [ ]:
# 看看区域的信用利差表现
df=province_credit_premium

def province_credit_premium_fig(df):

    dff=pd.merge(pd.DataFrame(df,columns=["信用利差"]),geo_data,left_on="区域",right_on="区域")

    fig = px.choropleth_mapbox(dff, geojson=gs_data, locations='id', color='信用利差',
            range_color=(20, 400),
            zoom=3, center = {"lat": 37.4189, "lon": 116.4219},
            mapbox_style='carto-positron',
            hover_data=["区域", "信用利差"]
            )

    fig.update_geos(fitbounds="locations", visible=True)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    return fig